In [1]:
# Hidden cell
import warnings

warnings.simplefilter('ignore')

# Compare atomic files

This notebook shows how to compare the tables `levels_prepared` and `lines_prepared` of the atomic files generated by Carsus.

## Create sample files

Let's create two different TARDIS atomic files.

In [2]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.chianti_ import ChiantiReader
from carsus.io.output import TARDISAtomData

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets


For the first atomic file we grab species `H-C` from GFALL and replace `H-He` from Chianti.

In [3]:
atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies('H-C')
gfall_reader = GFALLReader(ions='H-C')
chianti_reader = ChiantiReader(ions='H-He', collisions=True, priority=20)
zeta_data = KnoxLongZeta()

[carsus.io.nist.weightscomp][INFO] - Downloading data from the NIST Atomic Weights and Isotopic Compositions Database. (weightscomp.py:49)


[ carsus.io.nist.ionization][INFO] - Downloading ionization energies from the NIST Atomic Spectra Database. (ionization.py:65)


[    carsus.io.kurucz.gfall][WARNING] - A specific combination to identify unique levels from GFALL data has not been given. Defaulting to ["energy", "j"]. (gfall.py:86)


In [4]:
atom_data_a = TARDISAtomData(atomic_weights,
                             ionization_energies,
                             gfall_reader,
                             zeta_data,
                             chianti_reader)

[    carsus.io.kurucz.gfall][INFO] - Parsing GFALL: http://kurucz.harvard.edu/linelists/gfall/gfall.dat (gfall.py:139)


[     carsus.io.output.base][INFO] - Ingesting levels from Chianti. (base.py:255)


[     carsus.io.output.base][INFO] - Ingesting lines from GFALL. (base.py:327)


[    carsus.io.kurucz.gfall][INFO] - Extracting line data: atomic_number, ion_charge, energy_lower, j_lower, energy_upper, j_upper, wavelength, loggf. (gfall.py:343)


[     carsus.io.output.base][INFO] - Ingesting lines from Chianti. (base.py:352)


[     carsus.io.output.base][INFO] - Ingesting collisions from Chianti (base.py:499)


In [5]:
atom_data_a.to_hdf('A.h5')

[     carsus.io.output.base][INFO] - Signing TARDISAtomData. (base.py:840)


[     carsus.io.output.base][INFO] - MD5: 5c54bf15cb33253f18c4b0ba6c092182 (base.py:841)


[     carsus.io.output.base][INFO] - UUID1: b8e8bd78e25911eabc44c83dd46c2c53 (base.py:842)


For the first atomic file we grab species `H-C` from GFALL and replace `C` from Chianti.

In [6]:
chianti_reader = ChiantiReader(ions='C', collisions=True, priority=20)

In [7]:
atom_data_b = TARDISAtomData(atomic_weights,
                             ionization_energies,
                             gfall_reader,
                             zeta_data,
                             chianti_reader)

[     carsus.io.output.base][INFO] - Ingesting levels from Chianti. (base.py:255)


[     carsus.io.output.base][INFO] - Ingesting lines from GFALL. (base.py:327)


[     carsus.io.output.base][INFO] - Ingesting lines from Chianti. (base.py:352)


[     carsus.io.output.base][INFO] - Ingesting collisions from Chianti (base.py:499)


In [8]:
atom_data_b.to_hdf('B.h5')

[     carsus.io.output.base][INFO] - Signing TARDISAtomData. (base.py:840)


[     carsus.io.output.base][INFO] - MD5: 9e52816947227e29d7b653c9d47e2260 (base.py:841)


[     carsus.io.output.base][INFO] - UUID1: d50362d8e25911eabc44c83dd46c2c53 (base.py:842)


## Energy levels and transition lines

Define the following functions to compare both dataframes.

In [9]:
import pandas as pd
from carsus.util import parse_selected_species

In [10]:
def highlight_values(val):
    if val == True:
        return 'background-color: #BCF5A9'
    else:
        return 'background-color: #F5A9A9'
    
def highlight_diff(val):
    if val == 0:
        return 'background-color: #BCF5A9'
    else:
        return 'background-color: #F5A9A9'

In [11]:
def compare_levels_lines(path_a, path_b, ions='H-Zn'):
    
    # Read data
    levels_a = pd.read_hdf(path_a, key='levels')
    levels_b = pd.read_hdf(path_b, key='levels')
    lines_a = pd.read_hdf(path_a, key='lines')
    lines_b = pd.read_hdf(path_b, key='lines')
    
    # Get ions list
    ions = parse_selected_species(ions)
    
    lvl_eq = []
    lns_eq = []
    for ion in ions:
        
        # How many levels per ion in A
        try:
            num_lvl_a = len(levels_a.loc[ion])
            
        except (KeyError, TypeError, ValueError):
            num_lvl_a = 0
            
        # How many levels per ion in B
        try:
            num_lvl_b = len(levels_b.loc[ion])
            
        except (KeyError, TypeError, ValueError):
            num_lvl_b = 0

        # If level number is the same in A and B (and not zero) 
        # then compare cell against cell. `True` means all cells 
        # are equal in both dataframes.
        if num_lvl_a == num_lvl_b:
            val_lvl = True
            
            if num_lvl_a != 0:
                try:
                    k = levels_a.loc[ion].eq(levels_b.loc[ion]).sum().sum()
                    if num_lvl_a*3 != k:  # x3 because this df has three columns!
                        val_lvl = False
                        
                except (KeyError, TypeError, ValueError):
                    pass

        else:
            val_lvl = False
            
        # Append the results
        lvl_eq.append((ion, num_lvl_a, num_lvl_b, val_lvl))
        
        
        # Same for lines
        try:
            num_lns_a = len(lines_a.loc[ion])
            
        except (KeyError, TypeError, ValueError):
            num_lns_a = 0
            
        try:
            num_lns_b = len(lines_b.loc[ion])
            
        except (KeyError, TypeError, ValueError):
            num_lns_b = 0

        if num_lns_a == num_lns_b:
            val_lns = True
            
            if num_lns_a != 0:
                try:
                    k = lines_a.loc[ion].eq(lines_b.loc[ion]).sum().sum()
                    if num_lns_a*8 != k:
                        val_lvl = False
            
                except (KeyError, TypeError, ValueError):
                    pass
            
        else:
            val_lns = False
        
        lns_eq.append((ion, num_lns_a, num_lns_b, val_lns))
            
    df_lvl = pd.DataFrame(lvl_eq, columns=['ion', 'num_lvl_a', 'num_lvl_b', 'val_lvl'])
    df_lns = pd.DataFrame(lns_eq, columns=['ion', 'num_lns_a', 'num_lns_b', 'val_lns'])
    df = pd.merge(df_lvl, df_lns).set_index('ion')
    
    df['diff_lvl'] = abs(df['num_lvl_b'] - df['num_lvl_a'])
    df['diff_lns'] = abs(df['num_lns_b'] - df['num_lns_a'])
    df = df[['num_lvl_a', 'num_lvl_b', 'diff_lvl', 'val_lvl', 
          'num_lns_a', 'num_lns_b', 'diff_lns', 'val_lns']]

    return df

Then, comparison is straightforward.

In [12]:
tt = compare_levels_lines('A.h5', 'B.h5', ions='H-C')

- **num_xxx_x:** _(int)_ number of levels/lines.
- **diff_xxx:** _(int)_ difference in number of levels/lines.
- **val_xxx:** _(bool)_ _True_ if levels/lines have the same value.

In [13]:
tt.style.applymap(highlight_values, subset=['val_lvl', 'val_lns']).applymap(
                    highlight_diff, subset=['diff_lvl', 'diff_lns'])

,num_lvl_a,num_lvl_b,diff_lvl,val_lvl,num_lns_a,num_lns_b,diff_lns,val_lns
ion,,,,,,,,
"(1, 0)",25,1,24,False,74,0,74,False
"(2, 0)",49,744,695,False,174,605,431,False
"(2, 1)",25,9,16,False,67,7,60,False
"(3, 0)",67,67,0,True,403,403,0,True
"(3, 1)",55,55,0,True,135,135,0,True
"(3, 2)",1,1,0,True,0,0,0,True
"(4, 0)",28,28,0,True,39,39,0,True
"(4, 1)",39,39,0,True,185,185,0,True
"(4, 2)",17,17,0,True,27,27,0,True


In [14]:
# Hidden cell
!rm A.h5 B.h5